# Lab 2.2.4 *The Google BigQuery UI and API*

The Google BigQuery UI provides access to Google's extensive collection of public data sets via an SQL-based query engine.

The BigQuery API provides programmatic access to the data sets.

We can use the UI to discover interesting data before writing Python code to access it. Then we can reproduce it in an API request so as to aggregate large amounts of data on Google's infrastructure before pulling the results into our application.

# Creating a Google BigQuery Project & Service Account

1. Go to https://console.cloud.google.com/
  - Select the respective Google account.
2. Create a project:
  - Provide a project's name.
3. Create a Service Account:
  - Select "APIs & Services" -> "Credentials".
  - Click "+ CREATE CREDENTIALS" and select "Service Account".
  - Fill in "Service account name" and "Service account description".
  - Grant "Project Owner" role.
4. Create access key:
  - Select the Service Account.
  - Click "Manage service account".
  - Under "Options", click "Create key".
  - Select "JSON".
5. Store the JSON access key file.

# Creating a table on Google BigQuery

1. Select the desired project.
2. Create dataset:
   - Click "CREATE DATASET".
   - Fill "Dataset ID".
3. Create table:
   - Select dataset.
   - Click "+" to create a table.
   - Select "Source" for the data's source.
   - Under "Destination", select "Search for a project". 
   - Ensure "Project name" and "Dataset name" matches the respective info.
   - Click "Create table".

# Using the Python API

In [1]:
# import six
# print(six.__version__)

In [1]:
# python3 -m pip install -U six
# conda install -c conda-forge google-cloud-sdk
# conda install -c conda-forge google-cloud-storage
# conda install -c conda-forge google-cloud-bigquery

import google.cloud
from google.cloud import bigquery
from google.cloud import storage
# from google.oauth2 import service_account
from google.cloud.bigquery import Dataset
import json
import pandas as pd

KEY_PATH = 'double-platform-296512-461406cff561.json'

print("bigquery version: ", bigquery.__version__)
print("storage version: ", storage.__version__)

bigquery version:  2.4.0
storage version:  1.33.0


## Setup access via service account

In [2]:
client = bigquery.Client.from_service_account_json(r"C:\Users\Faizal Rahman\Downloads\tonal-works-297211-ce22506d867c.json")

# Accessing JL's Google Bigquery Project

In [3]:
PROJECT_ID = 'tonal-works-297211'
dataset_id = 'titanic'

dataset_ref = client.dataset(dataset_id, project=PROJECT_ID)
print(dataset_ref)

DatasetReference('tonal-works-297211', 'titanic')


In [4]:
print("The project's path: ", dataset_ref.path)

The project's path:  /projects/tonal-works-297211/datasets/titanic


## Titanic dataset

### Access to Titanic's dataset via dataset connection

In [5]:
project = "tonal-works-297211"
dataset_id = "titanic"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("titanic")
table = client.get_table(table_ref)
print("table: ", table)

table:  Table(TableReference(DatasetReference('tonal-works-297211', 'titanic'), 'titanic'))


In [6]:
df_titanic = client.list_rows(table).to_dataframe()
assert isinstance(df_titanic, pd.DataFrame)
assert len(list(df_titanic)) == len(table.schema)  # verify the number of columns
assert len(df_titanic) == table.num_rows  # verify the number of rows 

C:\Users\Faizal Rahman\anaconda3\lib\site-packages\google\cloud\bigquery\client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


#### Raw results from Titanic's dataset as a dataframe

In [7]:
df_titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,180,0,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0000,None,S
1,264,0,1,"Harrison, Mr. William",male,40.0,0,0,112059,0.0000,B94,S
2,278,0,2,"Parkes, Mr. Francis ""Frank""",male,NaN,0,0,239853,0.0000,None,S
3,303,0,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0000,None,S
4,414,0,2,"Cunningham, Mr. Alfred Fleming",male,NaN,0,0,239853,0.0000,None,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,456,1,3,"Jalsevac, Mr. Ivan",male,29.0,0,0,349240,7.8958,None,C
887,497,1,1,"Eustis, Miss. Elizabeth Mussey",female,54.0,1,0,36947,78.2667,D20,C
888,592,1,1,"Stephenson, Mrs. Walter Bertram (Martha Eustis)",female,52.0,1,0,36947,78.2667,D20,C
889,292,1,1,"Bishop, Mrs. Dickinson H (Helen Walton)",female,19.0,1,0,11967,91.0792,B49,C


# Accessing Google Bigquery's Public Data

In [8]:
bq_public_project = 'bigquery-public-data'
dataset_id = 'samples'

dataset_ref = client.dataset(dataset_id, project=bq_public_project)
print(dataset_ref)

DatasetReference('bigquery-public-data', 'samples')


In [9]:
print("The project's path: ", dataset_ref.path)

The project's path:  /projects/bigquery-public-data/datasets/samples


## Shakespeare's dataset

Reference: https://bigquery.cloud.google.com/table/bigquery-public-data:samples.shakespeare

### Access to Shakespeare's dataset via SQL query

In [10]:
query_string = '''
        SELECT corpus AS title, COUNT(*) AS unique_words
        FROM `publicdata.samples.shakespeare`
        GROUP BY title
        ORDER BY unique_words DESC
        LIMIT 10
        '''

query = client.query(query_string)

In [11]:
df_shakespeare1 = query.to_dataframe()
print(df_shakespeare1) 

                title  unique_words
0              hamlet          5318
1          kinghenryv          5104
2           cymbeline          4875
3  troilusandcressida          4795
4            kinglear          4784
5      kingrichardiii          4713
6        2kinghenryvi          4683
7          coriolanus          4653
8        2kinghenryiv          4605
9  antonyandcleopatra          4582


### Access to Shakespeare's dataset via dataset connection

In [12]:
project = "bigquery-public-data"
dataset_id = "samples"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("shakespeare")
table = client.get_table(table_ref)
print("table: ", table)

table:  Table(TableReference(DatasetReference('bigquery-public-data', 'samples'), 'shakespeare'))


In [13]:
df_shakespeare2 = client.list_rows(table).to_dataframe()
assert isinstance(df_shakespeare2, pd.DataFrame)
assert len(list(df_shakespeare2)) == len(table.schema)  # verify the number of columns
assert len(df_shakespeare2) == table.num_rows  # verify the number of rows 

C:\Users\Faizal Rahman\anaconda3\lib\site-packages\google\cloud\bigquery\client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


#### Raw results from Shakespeare's dataset as a dataframe

In [14]:
df_shakespeare2

,word,word_count,corpus,corpus_date
0,LVII,1,sonnets,0
1,augurs,1,sonnets,0
2,dimm'd,1,sonnets,0
3,plagues,1,sonnets,0
4,treason,1,sonnets,0
...,...,...,...,...
164651,that,232,kinghenryviii,1612
164652,his,234,kinghenryviii,1612
164653,me,235,kinghenryviii,1612
164654,your,242,kinghenryviii,1612


#### Count no. of unique words (*word*) per title (*corpus*)

In [15]:
df_shakespeare2.groupby(['corpus']).agg(['count'])

,word,word_count,corpus_date
,count,count,count
corpus,,,
1kinghenryiv,4317,4317,4317
1kinghenryvi,4441,4441,4441
2kinghenryiv,4605,4605,4605
2kinghenryvi,4683,4683,4683
3kinghenryvi,4076,4076,4076
allswellthatendswell,3949,3949,3949
antonyandcleopatra,4582,4582,4582
asyoulikeit,3676,3676,3676


In [16]:
# Filter for column "word" before aggregation
df_shakespeare2.groupby(['corpus'])['word'].agg(['count']).sort_values('count', ascending=False).reset_index()

,corpus,count
0,hamlet,5318
1,kinghenryv,5104
2,cymbeline,4875
3,troilusandcressida,4795
4,kinglear,4784
5,kingrichardiii,4713
6,2kinghenryvi,4683
7,coriolanus,4653
8,2kinghenryiv,4605
9,antonyandcleopatra,4582


In [17]:
# Filter for column "word" after aggregation
df_shakespeare2.groupby(['corpus']).agg(['count'])['word'].sort_values('count', ascending=False).reset_index()

,corpus,count
0,hamlet,5318
1,kinghenryv,5104
2,cymbeline,4875
3,troilusandcressida,4795
4,kinglear,4784
5,kingrichardiii,4713
6,2kinghenryvi,4683
7,coriolanus,4653
8,2kinghenryiv,4605
9,antonyandcleopatra,4582
